In [1]:
print("hello")

hello


Natural Language Processing Project

Tamil News classification

R. Shruthi
Shri Kumaran P A
Dasari Nagaveni

In [2]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import SpatialDropout1D
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from sklearn.feature_selection import RFE
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
train=pd.read_csv('tamil_news_train.csv')
train.head()
train.shape

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/May/19941-miss-iraq-competition-after-43-...,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்
1,2018/Dec/55921-viswasam-s-rustic-folk-song-has...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா
2,2018/Dec/55929-inspector-who-bought-insurance-...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு
3,2017/Jan/16161-morarji-desai-budget.txt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா
4,2018/Jun/46476-i-have-not-joined-bjp-or-anyoth...,“நானா? பாஜகவில் இணைந்துவிட்டேனா” - வரலட்சுமி வ...,cinema,சினிமா


(14521, 4)

In [4]:
np.random.seed(7)
train=train.drop_duplicates().reset_index(drop=True)
train.shape

(10579, 4)

In [5]:
train.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/May/19941-miss-iraq-competition-after-43-...,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்
1,2018/Dec/55921-viswasam-s-rustic-folk-song-has...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா
2,2018/Dec/55929-inspector-who-bought-insurance-...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு
3,2017/Jan/16161-morarji-desai-budget.txt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா
4,2018/Jun/46476-i-have-not-joined-bjp-or-anyoth...,“நானா? பாஜகவில் இணைந்துவிட்டேனா” - வரலட்சுமி வ...,cinema,சினிமா


In [6]:
test=pd.read_csv('tamil_news_test.csv')
test.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/Jun/21972-groom-chewing-gutka-during-wedd...,தாலி கட்ற நேரத்துலயும் குட்காவா? மணமகனை உதறினா...,india,இந்தியா
1,2018/Nov/54005-indvswi-3rd-t20-west-indies-won...,3வது டி20 : வெஸ்ட் இண்டீஸ் முதல் பேட்டிங் - வா...,sports,விளையாட்டு
2,2018/Dec/55960-lkg-ukg-classes-will-be-started...,"பெற்றோர்கள் விருப்பத்திற்கு ஏற்ப எல்.கே.ஜி., ய...",tamilnadu,தமிழ்நாடு
3,2018/Mar/41755-college-student-murder-in-chenn...,சென்னையில் கல்லூரி வாசலில் மாணவி குத்திக்கொலை,tamilnadu,தமிழ்நாடு
4,2018/Sep/51691-fever-in-uttar-pradesh-71-child...,உ.பி.யில் காய்ச்சலால் 45 நாட்களில் 71 குழந்தைக...,india,இந்தியா


In [7]:
test.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/Jun/21972-groom-chewing-gutka-during-wedd...,தாலி கட்ற நேரத்துலயும் குட்காவா? மணமகனை உதறினா...,india,இந்தியா
1,2018/Nov/54005-indvswi-3rd-t20-west-indies-won...,3வது டி20 : வெஸ்ட் இண்டீஸ் முதல் பேட்டிங் - வா...,sports,விளையாட்டு
2,2018/Dec/55960-lkg-ukg-classes-will-be-started...,"பெற்றோர்கள் விருப்பத்திற்கு ஏற்ப எல்.கே.ஜி., ய...",tamilnadu,தமிழ்நாடு
3,2018/Mar/41755-college-student-murder-in-chenn...,சென்னையில் கல்லூரி வாசலில் மாணவி குத்திக்கொலை,tamilnadu,தமிழ்நாடு
4,2018/Sep/51691-fever-in-uttar-pradesh-71-child...,உ.பி.யில் காய்ச்சலால் 45 நாட்களில் 71 குழந்தைக...,india,இந்தியா


In [8]:
qn='மச்சுபிச்சு மலை ரகசியம்'
st=qn.split()
st

['மச்சுபிச்சு', 'மலை', 'ரகசியம்']

In [9]:
category=''
maxi=0

In [10]:
import string
import re

In [11]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [12]:
def punctuation_remove(text):
    punctuation="".join([t for t in text if t not in string.punctuation])
    return punctuation
for i in range(len(train.NewsInTamil)):
    train.NewsInTamil[i]=punctuation_remove(train.NewsInTamil[i])
    train.NewsInEnglish[i]=punctuation_remove(train.NewsInEnglish[i])
train.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017May19941missiraqcompetitionafter43yearstxt,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்
1,2018Dec55921viswasamsrusticfolksonghasbeenrele...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா
2,2018Dec55929inspectorwhoboughtinsuranceathisow...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு
3,2017Jan16161morarjidesaibudgettxt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா
4,2018Jun46476ihavenotjoinedbjporanyotherpolitic...,“நானா பாஜகவில் இணைந்துவிட்டேனா” வரலட்சுமி விள...,cinema,சினிமா


In [13]:
def tokenisation(text):
    tokens=re.split(' ',text)
    return tokens

In [14]:
train.CategoryInTamil.unique()

array([' உலகம்', ' சினிமா', ' தமிழ்நாடு', ' இந்தியா', ' அரசியல்',
       ' விளையாட்டு'], dtype=object)

In [15]:
stop_words=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']


In [16]:
def stopwordsremove(text):
    removed=[s for s in text if s not in stop_words]
    return removed
print(tokenisation(train.NewsInTamil[0]))

['ஈராக்கில்', '43', 'ஆண்டுகள்', 'கழித்து', 'அழகிப்போட்டி']


In [17]:
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [18]:
qn=stopwordsremove(tokenisation(punctuation_remove(test.NewsInTamil[60])))
qn

['ரஜினி', 'அரசியல்', 'புயலா', 'புஸ்வாணமா']

In [28]:
maxi=-1
category=''
for i in range(len(train.NewsInTamil)):
    counts=0
    check=stopwordsremove(tokenisation(train.NewsInTamil[i]))
    for k in qn:
        for m in check:
            if(k==m):
                counts=counts+1
    if(counts>maxi):
        maxi=counts
        category=train.CategoryInTamil[i]
print(category)

 அரசியல்


In [19]:
maxi=-1
category=''
test_list=[]
for j in range(100):
    check_test=stopwordsremove(tokenisation(test.NewsInTamil[j]))
    maxi=-1
    for i in range(len(train.NewsInTamil)):
        counts=0
        check=stopwordsremove(tokenisation(train.NewsInTamil[i]))
        for k in check_test:
            for m in check:
                if(k==m):
                    counts=counts+1
        if(counts>maxi):
            maxi=counts
            category=train.CategoryInTamil[i]
    test_list.append(category)
test_list

[' தமிழ்நாடு',
 ' விளையாட்டு',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' இந்தியா',
 ' தமிழ்நாடு',
 ' சினிமா',
 ' விளையாட்டு',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' இந்தியா',
 ' இந்தியா',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' அரசியல்',
 ' தமிழ்நாடு',
 ' அரசியல்',
 ' தமிழ்நாடு',
 ' சினிமா',
 ' சினிமா',
 ' சினிமா',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' விளையாட்டு',
 ' விளையாட்டு',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' விளையாட்டு',
 ' அரசியல்',
 ' இந்தியா',
 ' இந்தியா',
 ' இந்தியா',
 ' இந்தியா',
 ' சினிமா',
 ' சினிமா',
 ' விளையாட்டு',
 ' சினிமா',
 ' இந்தியா',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' தமிழ்நாடு',
 ' சினிமா',
 ' விளையாட்டு',
 ' விளையாட்டு',
 ' சினிமா',
 ' இந்தியா',
 ' விளையாட்டு',
 ' உலகம்',
 ' அரசியல்',
 ' சினிமா',
 ' விளையாட்டு',
 ' சினிமா',
 ' தமிழ்நாடு',
 ' இந்தியா',
 ' இந்தியா',
 ' இந்தியா',
 ' உலகம்',
 ' இந்தியா',
 ' சினிமா',
 ' இந்தியா',
 ' விளையாட்டு',
 ' இந்தியா',
 ' சினிமா',
 ' சினிமா',
 ' தமிழ்நாடு',
 ' விளையாட்டு',
 ' தமிழ்நாடு',
 ' விளையாட்டு',
 ' சினிமா',
 ' இந்தியா',
 ' தமிழ்நாடு',

In [20]:
train_list=test.CategoryInTamil
c=0
i=0
for i in range(100):
    if(train_list[i]==test_list[i]):
        c=c+1
accuracy=c/100
accuracy

0.74

In [21]:
data=train

In [22]:
maxi=-1
category=''
newcol=[]
for i in range(len(train.NewsInTamil)):
    counts=0
    check=stopwordsremove(tokenisation(train.NewsInTamil[i]))
    for k in qn:
        counts=0
        for m in check:
            if(k==m):
                counts=counts+1
    newcol.append(counts)
    if(counts>maxi):
        maxi=counts
        category=train.CategoryInTamil[i]
data['count']=newcol

In [23]:
data.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil,count
0,2017May19941missiraqcompetitionafter43yearstxt,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்,0
1,2018Dec55921viswasamsrusticfolksonghasbeenrele...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா,0
2,2018Dec55929inspectorwhoboughtinsuranceathisow...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு,0
3,2017Jan16161morarjidesaibudgettxt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா,0
4,2018Jun46476ihavenotjoinedbjporanyotherpolitic...,“நானா பாஜகவில் இணைந்துவிட்டேனா” வரலட்சுமி விள...,cinema,சினிமா,0


In [24]:
xtrain=data[['count','CategoryInTamil']]
xtrain.head()
X=xtrain[['count']]
Y=xtrain[['CategoryInTamil']]

,count,CategoryInTamil
0,0,உலகம்
1,0,சினிமா
2,0,தமிழ்நாடு
3,0,இந்தியா
4,0,சினிமா


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt 

In [26]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.3,random_state=0)
clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(Xtrain,Ytrain)
y_pred=clf.predict([[maxi]])
y_pred

c:\Users\Lenovo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([' தமிழ்நாடு'], dtype=object)

In [27]:
world=train.loc[train['CategoryInTamil']==' உலகம்']
cinema=train.loc[train['CategoryInTamil']==' சினிமா']
tamilnadu=train.loc[train['CategoryInTamil']==' தமிழ்நாடு']
india=train.loc[train['CategoryInTamil']==' இந்தியா']
sports=train.loc[train['CategoryInTamil']==' விளையாட்டு']
politics=train.loc[train['CategoryInTamil']==' அரசியல்']

In [28]:
cinema.reset_index(inplace=True,drop=True)
world.reset_index(inplace=True,drop=True)
politics.reset_index(inplace=True,drop=True)
sports.reset_index(inplace=True,drop=True)
tamilnadu.reset_index(inplace=True,drop=True)
india.reset_index(inplace=True,drop=True)

In [29]:
cinema_words=[]
for i in range(0,len(cinema)):
    check=stopwordsremove(tokenisation(cinema.NewsInTamil[i]))
    for k in check:
        if k not in cinema_words:
            cinema_words.append(k)

In [30]:
world_words=[]
for i in range(0,len(world)):
    check=stopwordsremove(tokenisation(world.NewsInTamil[i]))
    for k in check:
        if k not in world_words:
            world_words.append(k)

In [31]:
india_words=[]
for i in range(0,len(india)):
    check=stopwordsremove(tokenisation(india.NewsInTamil[i]))
    for k in check:
        if k not in india_words:
            india_words.append(k)

In [32]:
politics_words=[]
for i in range(0,len(politics)):
    check=stopwordsremove(tokenisation(politics.NewsInTamil[i]))
    for k in check:
        if k not in politics_words:
            politics_words.append(k)

In [33]:
tamilnadu_words=[]
for i in range(0,len(tamilnadu)):
    check=stopwordsremove(tokenisation(tamilnadu.NewsInTamil[i]))
    for k in check:
        if k not in tamilnadu_words:
            tamilnadu_words.append(k)

In [34]:
sports_words=[]
for i in range(0,len(sports)):
    check=stopwordsremove(tokenisation(sports.NewsInTamil[i]))
    for k in check:
        if k not in sports_words:
            sports_words.append(k)

In [35]:
ypred=[]
for i in range(0,len(test)):
    check=stopwordsremove(tokenisation(test.NewsInTamil[i]))
    ci=0
    ct=0
    cs=0
    cc=0
    cp=0
    cw=0
    for k in check:
        if(k in world_words):
            cw=cw+1
        elif(k in cinema_words):
            cc=cc+1
        elif(k in india_words):
            ci=ci+1
        elif(k in sports_words):
            cs=cs+1
        elif(k in politics_words):
            cp=cp+1
        elif(k in tamilnadu_words):
            ct=ct+1
    if(cw>cc and cw>ci and cw>cs and cw>ct and cw>cp):
        ypred.append('உலகம்')
    elif(cc>cw and cc>ci and cc>cs and cc>ct and cc>cp):
        ypred.append('சினிமா')
    elif(ci>cw and ci>cc and ci>cs and ci>cp and ci>ct):
        ypred.append('இந்தியா')
    elif(cs>cw and cs>ci and cs>cp and cs>ct and cs>cc):
        ypred.append('விளையாட்டு')
    elif(ct>cw and ct>cs and ct>ci and ct>cc and ct>cp):
        ypred.append('தமிழ்நாடு')
    else:
        ypred.append('அரசியல்')
ypred

['தமிழ்நாடு',
 'விளையாட்டு',
 'தமிழ்நாடு',
 'அரசியல்',
 'அரசியல்',
 'அரசியல்',
 'சினிமா',
 'விளையாட்டு',
 'அரசியல்',
 'உலகம்',
 'இந்தியா',
 'உலகம்',
 'உலகம்',
 'இந்தியா',
 'அரசியல்',
 'உலகம்',
 'சினிமா',
 'தமிழ்நாடு',
 'உலகம்',
 'இந்தியா',
 'உலகம்',
 'சினிமா',
 'சினிமா',
 'உலகம்',
 'இந்தியா',
 'உலகம்',
 'விளையாட்டு',
 'அரசியல்',
 'உலகம்',
 'உலகம்',
 'அரசியல்',
 'அரசியல்',
 'இந்தியா',
 'உலகம்',
 'உலகம்',
 'அரசியல்',
 'சினிமா',
 'உலகம்',
 'சினிமா',
 'உலகம்',
 'உலகம்',
 'உலகம்',
 'இந்தியா',
 'அரசியல்',
 'அரசியல்',
 'அரசியல்',
 'சினிமா',
 'உலகம்',
 'தமிழ்நாடு',
 'இந்தியா',
 'உலகம்',
 'அரசியல்',
 'சினிமா',
 'சினிமா',
 'உலகம்',
 'இந்தியா',
 'அரசியல்',
 'அரசியல்',
 'உலகம்',
 'அரசியல்',
 'சினிமா',
 'இந்தியா',
 'விளையாட்டு',
 'சினிமா',
 'சினிமா',
 'சினிமா',
 'சினிமா',
 'விளையாட்டு',
 'அரசியல்',
 'உலகம்',
 'சினிமா',
 'இந்தியா',
 'உலகம்',
 'உலகம்',
 'அரசியல்',
 'அரசியல்',
 'தமிழ்நாடு',
 'இந்தியா',
 'உலகம்',
 'உலகம்',
 'உலகம்',
 'உலகம்',
 'விளையாட்டு',
 'உலகம்',
 'அரசியல்',
 'இந்தியா',
 'இந்தியா',

In [36]:
c=0
i=0
for i in range(len(test)):
    if(ypred[i]==test.CategoryInTamil[i]):
        c=c+1
accuracy=c/len(test)
accuracy

0.1
